In [1]:
import numpy as np
import plotly.graph_objects as go
import os
import gzip
import pickle as pkl

def loadall_results(path, horizon, n_folds):
    result = np.zeros( (n_folds, 19999) )
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):
            try:
                res = pkl.load(f)[:19999]
                result[i] = res
            except EOFError:
                break
    return result


In [ ]:

n_folds = 96
horizon = 20000

game = 'AT'
task = 'balanced'

colors = [  [250,0,0], [250,0,0],[250,0,0], [250,0,0], [250,0,0],
            [0,250,0], [0,250,0], [0,250,0], [0,250,0], [0,250,0],
            [0,0,250], [0,0,250], [0,0,250], [0,0,250],[0,0,250],
            [255,128,0], [255,128,0], [255,128,0], [255,128,0], [255,128,0]  ]

labels = [   'RandCBP_10_5', 'RandCBP_1_5',  'RandCBP_18_5', 'RandCBP_116_5','RandCBP_132_5', 
             'RandCBP_10_10', 'RandCBP_1_10',  'RandCBP_18_10', 'RandCBP_116_10','RandCBP_132_10',
             'RandCBP_10_20', 'RandCBP_1_20',  'RandCBP_18_20', 'RandCBP_116_20','RandCBP_132_20', 
             'RandCBP_10_100', 'RandCBP_1_100',  'RandCBP_18_100', 'RandCBP_116_100','RandCBP_132_100'  ]
             
fig1 = go.Figure(    )


for color, label in zip( colors, labels):
    r,g,b = color

    direct = './results/benchmark_randcbp/{}/'.format(game)
    path = os.path.join(direct, 'cr_{}_{}_{}_{}.pkl.gz'.format(task,horizon,n_folds,label) )
    result = loadall_results(path, horizon, n_folds)

    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std(result,0) 
    upper_regret = regret + std

    parts = label.rsplit('_', 1)
    first_part = parts[0]
    if 'RandCBP_132' == first_part:
        fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dot' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1/32'.format( label.split('_')[2] ) ) ) 

    elif 'RandCBP_116' == first_part:
        fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'longdashdot' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1/16'.format( label.split('_')[2] ) ) )  
    
    elif 'RandCBP_18' == first_part:
        fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dash' ), mode='lines',
                legendgroup='M = {}'.format( label.split('_')[2] ),  name='σ = 1/8'.format( label.split('_')[2] ) ) ) 
    
    elif 'RandCBP_1' == first_part:
        fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1'.format( label.split('_')[2] ) ) ) 
    
    elif 'RandCBP_10' == first_part:
        fig1.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'longdash' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ),legendgrouptitle_text='M = {}'.format( label.split('_')[2] ), name='σ = 10'.format( label.split('_')[2] ) ) ) 
    
    fig1.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.05)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False ) )

if game == 'AT' and task == 'balanced':
    fig1.update_yaxes(range=[0, 100] )
elif game == 'AT' and task == 'imbalanced':
    fig1.update_yaxes(range=[0, 15] )
elif game == 'LE' and task == 'imbalanced':
    fig1.update_yaxes(range=[0, 20] )
elif game == 'LE' and task == 'balanced':
    fig1.update_yaxes(range=[0, 500] )

fig1.update_layout(legend= dict(yanchor="top",y=1,xanchor="left",x=0.43),
                    legend_orientation="h",
                    autosize=False,
                    xaxis_title="Sequence",
                    yaxis_title="Regret",
                    margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                    font=dict(size=13,) )
fig1.show()
fig1.write_image("./benchmarkRandCBP_{}_{}.pdf".format(task, game) )


In [18]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import ttest_ind,mannwhitneyu

dfs = []

for game in ['AT', 'LE']:

    for task in ['imbalanced', 'balanced']:


        labels = [       'RandCBP_10_5', 'RandCBP_1_5',  'RandCBP_18_5', 'RandCBP_116_5','RandCBP_132_5', 
                         'RandCBP_10_10', 'RandCBP_1_10',  'RandCBP_18_10', 'RandCBP_116_10','RandCBP_132_10',
                         'RandCBP_10_20', 'RandCBP_1_20',  'RandCBP_18_20', 'RandCBP_116_20','RandCBP_132_20', 
                         'RandCBP_10_100', 'RandCBP_1_100',  'RandCBP_18_100', 'RandCBP_116_100','RandCBP_132_100'  ]

        perf_dic = {}

        for label in  labels:

            r,g,b = color
                
            direct = './results/benchmark_randcbp/{}/'.format(game)
            path = os.path.join(direct, 'cr_{}_20000_96_{}.pkl.gz'.format(task, label) )
            result = loadall_results(path, horizon, n_folds)

            perf_dic[label] = result[:,-1] 

        df = pd.DataFrame.from_dict(perf_dic)

        ############### create data frame:

        # Calculate mean and std
        mean_values = df.mean()
        std_values = df.std()

        # Calculate p-values
        p_values = {}
        for strategy in df.columns:
            if strategy == 'RandCBP_10_5':
                p =  1
                p_values[strategy] = p
            else:
                s, p = ttest_ind(df['RandCBP_10_5'], df[strategy], equal_var=False)
                # s, p = mannwhitneyu(df['RandCBP'], df[strategy],  alternative='less')
                # s, p = ttest_ind( df[strategy], df['RandCBP'])
                p_values[strategy] = p

        # Find the minimum value within each row
        min_values = df.min(axis=1)
        # Create a boolean DataFrame indicating if each strategy achieved the minimum value
        strategy_won = df.eq(min_values, axis=0)
        # Count how many times each strategy won
        winning_counts = strategy_won.sum()
        # winning_counts = df.apply(idxmin_excluding_duplicates, axis=1).value_counts()

        # Create the new DataFrame
        new_data = {
            'Mean_{}_{}'.format(game,task): mean_values,
            'Std_{}_{}'.format(game,task): std_values,
            'PValue_{}_{}'.format(game,task): p_values,
            'WinningCount_{}_{}'.format(game,task): winning_counts,  }

        pd.options.display.float_format = '{:.3f}'.format
        
        new_df = pd.DataFrame(new_data)
        new_df['WinningCount_{}_{}'.format(game,task)].fillna(0, inplace=True)
        new_df['WinningCount_{}_{}'.format(game,task)] = new_df['WinningCount_{}_{}'.format(game,task)].astype(int) 
        dfs.append(new_df)

In [19]:
df = pd.concat(dfs, axis=1)
# df = df.round(2)

# Sample DataFrame columns
columns = [
    'Mean_AT_imbalanced', 'Std_AT_imbalanced', 'WinningCount_AT_imbalanced', 'PValue_AT_imbalanced',
    'Mean_AT_balanced', 'Std_AT_balanced', 'Winning Count_AT_balanced', 'P-Value_AT_balanced',
    'Mean_LE_imbalanced', 'Std_LE_imbalanced', 'WinningCount_LE_imbalanced', 'PValue_LE_imbalanced',
    'Mean_LE_balanced', 'Std_LE_balanced', 'Winning Count_LE_balanced', 'P-Value_LE_balanced'
]

# Create arrays for each level of the multi-index
first_level = ['mean', 'std', 'pvalue', 'win count',] * 4
second_level = ['imbalanced','imbalanced','imbalanced','imbalanced', 'balanced', 'balanced', 'balanced', 'balanced'] * 2
third_level = ['AT', 'AT', 'AT', 'AT','AT', 'AT', 'AT', 'AT', 'LE', 'LE', 'LE', 'LE', 'LE', 'LE', 'LE', 'LE'] 

# Create a MultiIndex from the arrays
multi_index = pd.MultiIndex.from_arrays([third_level , second_level, first_level], names=('Game', 'Case', 'Metric'))

# Create a DataFrame with the specified columns and multi-index
# df = pd.DataFrame(columns=columns, index=[0])
df.columns = multi_index

# desired_order = [ 'RandCBP', 'CBP', 'PM_DMED',  'TSPM_1', 'TSPM_0', 'BPM_LEAST',   ]
# df = df.reindex(desired_order)

# df.index = [ 'RandCBP', 'CBP', 'PM-DMED',  'TSPM', 'TSPM Gaussian', 'BPM-Least',   ]

df = df.round(3)

df


Game                    AT                                                     \
Case            imbalanced                           balanced                   
Metric                mean      std pvalue win count     mean      std pvalue   
RandCBP_10_5       920.092 3601.543  1.000        90  753.398 1195.065  1.000   
RandCBP_1_5          4.689    4.070  0.014        39   41.417   78.310  0.000   
RandCBP_18_5         4.453    4.004  0.014        41   40.615   75.230  0.000   
RandCBP_116_5        4.379    3.820  0.014        41   41.450   75.994  0.000   
RandCBP_132_5        4.379    3.820  0.014        41   41.355   76.254  0.000   
RandCBP_10_10      920.092 3601.543  1.000        90  704.156 1190.377  0.775   
RandCBP_1_10         5.607    4.795  0.015        40   43.368   66.237  0.000   
RandCBP_18_10        5.445    4.542  0.015        42   79.007  248.019  0.000   
RandCBP_116_10       5.526    4.706  0.015        42   76.933  248.009  0.000   
RandCBP_132_10       5.526    4.706  0.015        42   81.401  249.446  0.000   
RandCBP_10_20      920.092 3601.543  1.000        90  701.259 1141.956  0.758   
RandCBP_1_20         6.124    5.251  0.015        41   45.195   72.152  0.000   
RandCBP_18_20        6.078    5.220  0.015        42   58.687  101.028  0.000   
RandCBP_116_20       6.046    5.206  0.015        42   62.951  106.077  0.000   
RandCBP_132_20       6.046    5.206  0.015        42   62.494  105.635  0.000   
RandCBP_10_100     920.092 3601.543  1.000        90  750.504 1196.308  0.987   
RandCBP_1_100        6.723    5.753  0.015        41   44.336   70.878  0.000   
RandCBP_18_100       6.629    5.714  0.015        44   65.256  102.279  0.000   
RandCBP_116_100      6.714    5.856  0.015        44   61.851  103.987  0.000   
RandCBP_132_100      6.714    5.856  0.015        44   63.074  105.438  0.000   

Game                              LE                                     \
Case                      imbalanced                           balanced   
Metric          win count       mean      std pvalue win count     mean   
RandCBP_10_5           60   1522.058 4523.762  1.000        86 1071.020   
RandCBP_1_5            14     11.981   10.148  0.001         1  315.303   
RandCBP_18_5           19     10.756    7.630  0.001         6  359.850   
RandCBP_116_5          18     10.847    7.767  0.001         6  338.292   
RandCBP_132_5          18     10.847    7.767  0.001         6  351.907   
RandCBP_10_10          50   1522.058 4523.762  1.000        86 1071.020   
RandCBP_1_10           14     13.779   23.142  0.002         3  322.168   
RandCBP_18_10          19     12.932    6.678  0.002         2  476.500   
RandCBP_116_10         19     12.834    6.779  0.002         2  456.015   
RandCBP_132_10         19     12.888    6.786  0.002         2  459.441   
RandCBP_10_20          50   1522.058 4523.762  1.000        86 1071.020   
RandCBP_1_20           15     14.688   25.046  0.002         2  330.992   
RandCBP_18_20          24     14.165    8.090  0.002         1  491.336   
RandCBP_116_20         23     14.216    7.980  0.002         1  482.239   
RandCBP_132_20         23     14.216    7.980  0.002         1  490.132   
RandCBP_10_100         48   1522.058 4523.762  1.000        86 1071.020   
RandCBP_1_100          18     14.299   23.416  0.002         1  352.404   
RandCBP_18_100         31     16.012   10.813  0.002         0  482.535   
RandCBP_116_100        32     15.993   10.777  0.002         0  489.491   
RandCBP_132_100        33     15.975   10.787  0.002         0  484.960   

Game                                       
Case                                       
Metric               std pvalue win count  
RandCBP_10_5    1330.973  1.000        52  
RandCBP_1_5      459.127  0.000         6  
RandCBP_18_5     367.508  0.000        10  
RandCBP_116_5    372.219  0.000         7  
RandCBP_132_5    378.420  0.000         7  
RandCBP_10_10   1330.973  1.000        52  
RandCBP_1_10  

In [20]:
df.to_csv('dataframe_randcbp.csv')